# Mempersiapkan environment

##a. Install pyspark package
Untuk menjalankan notebook ini di Google colab, langkah pertama yang perlu dilakukan adalah menginstall package `pyspark`, karena package tersebut tidak disertakan secara default.

Langkah ini perlu dilakukan setiap membuka session/notebook baru.

Instalasi kita lakukan dengan perintah `pip`

In [ ]:
%pip install pyspark


##b. Membuat spark session

Import library yang akan digunakan.

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

Inisialisasi spark session untuk berinteraksi dengan Spark cluster

In [ ]:
spark = SparkSession.builder.appName('DataFrame Basics').getOrCreate()

#1.Lab 02 - Membuat DataFrame

DataFrame dapat dibuat dengan banyak cara, di antaranya :
- Dari python object, misalnya array/list, dictionary, pandas dataframe, dll
- Dari file : csv, json, dll
- Dari HDFS
- Dari RDD
- dll.

##1.1. Create From Array

In [ ]:
mydata = (('DKI JAKARTA',15328),
('JAWA BARAT',1320),
('JAWA TENGAH',1030),
('DI YOGYAKARTA',1174),
('JAWA TIMUR',813),
('BANTEN',1237))

df_from_array = spark.createDataFrame(mydata).toDF("province", "density")

df_from_array.show()

##1.2. Create from Pandas DataFrame

Dowload file

In [ ]:
!wget https://www.dropbox.com/s/65mohvoyjshtqa9/penduduk2015.csv

Create pandas dataframe dari file csv tersebut

In [ ]:
import pandas as pd

pddf = pd.read_csv('penduduk2015.csv')
pddf

Ubah ke Spark dataframe

In [ ]:
df_from_pandas = spark.createDataFrame(pddf)
df_from_pandas.show()

##1.3. Create from csv

Kita juga bisa me-load langsung file csv tersebut ke Spark dataframe

In [ ]:
df_from_csv = spark.read.csv("penduduk2015.csv", header=True)
df_from_csv.show()

##1.4. Create from JSON

In [ ]:
!wget https://www.dropbox.com/s/8wpyzukkmmn3t6g/penduduk2015.json

Tampilkan isi file dengan perintah `cat`

In [ ]:
!cat penduduk2015.json

Untuk membaca multiline JSON, set parameter `multiline` = True

In [ ]:
dfj = spark.read.json("penduduk2015.json", multiLine=True)
dfj.show()

#2.Lab 03 - Explorasi DataFrame

Dalam latihan ini kita akan mencoba berbagai operasi pada Spark DataFrame untuk melakukan eksplorasi data.

Kita akan menggunakan data kepadatan penduduk per propinsi.

In [ ]:
!wget https://www.dropbox.com/s/4ozzaa2yvy2kkza/indonesia2013-2015.csv

Kita gunakan magic command untuk melihat ukuran dan isi file (karena file kita cukup kecil).

In [ ]:
%ls -al indonesia2013-2015.csv

In [ ]:
%cat indonesia2013-2015.csv

Karena data yang kita load sudah bersih, kita akan set inferSchema = True agar Spark menyesuaikan tipe kolom dengan datanya.

In [ ]:
df = spark.read.csv("indonesia2013-2015.csv",header=True,inferSchema=True)

###2.1 Melihat sekilas

####Menampilkan beberapa baris

Biasanya kita menampilkan beberapa baris data untuk mengecek format dan konten dataframe yang kita buat.

Untuk menampilkan beberapa baris dari dataframe, kita bisa gunakan perintah ``show(n)`` untuk menampilkan n baris pertama, atau ``first()`` untuk menampilkan 1 baris pertama saja.

In [ ]:
df.show(5)
df.first()

####Menampilkan jumlah kolom

In [ ]:
len(df.columns)

####Menampilkan total records

In [ ]:
df.count()

####Menampilkan skema

Untuk menampilkan skema dataframe, gunakan fungsi `printSchema()`



In [ ]:
df.printSchema()

Akses atribut `columns` untuk menampilkan list nama kolom


In [ ]:
df.columns

Akses atribut `dtypes` untuk menampilkan list nama kolom beserta data type masing-masing kolom tersebut

In [ ]:
df.dtypes

####Menampilkan summary statistik

Fungsi `describe()` digunakan untuk menampilkan summary statistik dari seluruh kolom.

Jangan lupa memanggil fungsi `show()` untuk menampilkan hasilnya.

In [ ]:
df.describe().show()

Untuk menampilkan statistik dari salah satu kolom saja, gunakan nama kolom yang akan ditampilkan sebagai parameter. Misalnya `describe('column1')`

In [ ]:
df.describe("density").show()

###2.2 Filtering



Kita dapat melakukan filtering terhadap spark dataframe, berdasar kolom atau baris

####Memilih kolom tertentu

Untuk menampilkan kolom tertentu, digunakan fungsi `select('nama_kolom')`


In [ ]:
df.select("year").show(10)

Untuk memilih beberapa kolom, gunakan tanda koma sebagai pemisah

In [ ]:
df.select("province","density").show(10)

####Memilih records / baris

Untuk memilih baris dengan kondisi tertentu, gunakan fungsi `filter(<kondisi>)`



In [ ]:
df.filter(df.density > 5000).show()

Untuk menggunakan kondisi berupa operasi string, dapat digunakan fungsi-fungsi dari `pyspark.sql.Column` yang terkait string, misalnya `contains()`, `startswith()`, `endswith()`

In [ ]:
df.filter(df.province.contains('TENGGARA')).show(5)
df.filter(df.province.startswith('SU')).show(5)
df.filter(df.province.endswith('BARAT')).show(5)

Tersedia juga fungsi `like()` yang serupa dengan SQL statement *like*

In [ ]:
df.filter(df.province.like('SU%')).show(5)

Atau dapat juga menggunakan regex, dengan fungsi `rlike()`

In [ ]:
df.filter(df.province.rlike('[A-Z]*TA$')).show()

Dapat juga menggunakan filter berdasar list, dengan fungsi `isin()`

In [ ]:
df.filter(df.timezone.isin('WIT','WITA')).show(5)

Untuk menggunakan beberapa kondisi sekaligus, menggunakan tanda `&` untuk AND dan `|` untuk OR, dengan masing-masing kondisi dilingkupi tanda kurung `()`

In [ ]:
df.filter((df.timezone.isin('WIT','WITA')) & (df.year == 2013)).show(5)

###2.3 Unique value

Untuk menampilkan nilai unik dari dataframe, digunakan fungsi `distinct()`.

Nilai unik di sini adalah kombinasi nilai dari seluruh kolom.

In [ ]:
df.distinct().show()

Untuk menampilkan nilai unik dari kolom tertentu, tulis nama kolom yang dimaksud sebagai parameter.

In [ ]:
df.select('timezone').distinct().show()
df.select('year','timezone').distinct().show()

####Menghapus duplikasi data

Untuk menghapus record duplikat, gunakan `dropDuplicates(subset)` atau `drop_duplicates(subset)`.

In [ ]:
df.dropDuplicates(['province', 'timezone']).show(100)

###2.4 Agregasi

####Group by column

Untuk mengelompokkan berdasar kolom, gunakan perintah `groupBy('nama_kolom')`

Untuk mengelompokkan berdasar lebih dari 1 kolom, gunakan tanda koma sebagai pemisah nama kolom.

In [ ]:
df.groupBy("timezone")

In [ ]:
df.groupBy("timezone","year")

Perintah `groupBy()` menghasilkan obyek `GroupedData` yang belum bisa ditampilkan.

Biasanya setelah pengelompokan, kita melakukan operasi sumarisasi data. Kita terapkan operasi tersebut pada objek hasil groupBy dengan memanggil fungsi yang dibutuhkan. Misalnya `count()` atau `max('namakolom')`

In [ ]:
df.groupBy('timezone').count().show()
df.groupBy('timezone').max('density').show()

Kita juga bisa menggunakan fungsi `agg()` untuk melakukan agregasi. Terutama jika kita ingin melakukan lebih dari 1 operasi agregat.

Kita bisa menggunakan fungsi `alias()` untuk memberi nama kolom hasil agregasi.

In [ ]:
df.groupBy("timezone").agg(F.max('density')).show()
df.groupBy("timezone").agg(F.avg('density').alias('avg_density'), \
                           F.min('density').alias('min_density'), \
                           F.max('density').alias('max_density')).show()

####Order By

In [ ]:
df.groupBy("timezone") \
  .mean("density") \
  .orderBy("timezone",ascending=False).show()

####Agregasi dengan filter / kondisi

In [ ]:
df.groupBy("timezone") \
  .mean("density") \
  .where(df.timezone.contains('WIT')).show()

####Filter hasil agregat (SQL stat **HAVING**)

Untuk memfilter berdasar hasil agregasi (semacam perintah **HAVING** di SQL), lakukan dalam 2 langkah.
1. Lakukan `groupBy` + `agg` dan beri nama kolom hasil agregat dengan `alias`
2. gunakan fungsi `filter(kondisi)` pada kolom hasil agregasi.

In [ ]:
df_agg = df.groupBy("timezone", "province") \
  .agg(F.avg("density").alias("avg_density")) \
  .where(df.timezone.contains('WIT'))

df_agg.filter(df_agg.avg_density > 50).show()

###2.5 Transformasi DataFrame

####Kolom baru berupa nilai konstan

Untuk menambahkan kolom baru ke dalam dataframe, kita bisa menggunakan perintah `withColumn()`

Sedangkan untuk menambahkan sebuah nilai konstan, kita bisa menggunakan fungsi `lit(nilai_konstan)` dari `pyspark.sql.functions`, yang berfungsi membuat kolom dari nilai literal/konstan.

Misalnya kita ingin menambahkan kolom **status** yang bernilai 1

In [ ]:
df.withColumn('status', F.lit(1)).show(5)

####Kolom baru dari kolom yang ada



In [ ]:
df.withColumn('tahun-1', df.year-1).show(5)

####Kondisional

Untuk menambahkan kolom berdasar beberapa kondisi, gunakan `when` dan `otherwise` (jika perlu).

Perhatikan bahwa `when` yang pertama adalah fungsi dalam `pyspark.sql.functions`, sedangkan `when` yang berikutnya adalah fungsi `when` pada object kolom (`pyspark.sql.Column`)

Fungsi `otherwise` adalah kondisi *else* atau kondisi selain yang disebutkan pada *when*.

In [ ]:
df.withColumn('timezone1', F.when(df.timezone == 'WIT', 1).
              when(df.timezone == 'WIT', 2).
              otherwise(3)).show(5)

###2.6 Data enrichment - Join


Perintah ntuk melakukan join adalah sebagai berikut :

`df1.join(df2, on=[columname], how=’left’)`

Where :
-    `df1` − Dataframe1.
-    `df2` – Dataframe2.
-    `on` − nama kolom yang akan digunakan untuk join.
-    `how` – type of join needs to be performed – `left`, `right`, `outer`, `inner`. Defaultnya adalah inner join.


Create dataframe yang akan digunakan untuk join

In [ ]:
damdata = (("SUMATERA SELATAN",2),
("SULAWESI TENGAH",2),
("SULAWESI SELATAN",2),
("SUMATERA BARAT",3),
("RIAU",3),
("LAMPUNG",3),
("NUSA TENGGARA TIMUR",4),
("BENGKULU",8),
("SUMATERA UTARA",10),
("JAWA TIMUR",12),
("JAWA TENGAH",35),
("JAWA BARAT",49))

df_dam = spark.createDataFrame(damdata).toDF("province", "dam_num")

df_dam.show()

Join dataframe kepadatan penduduk dengan dataframe jumlah bendungan, berdasarkan nama propinsi.

In [ ]:
df.join(df_dam, on=['province'], how='inner').show()

Untuk melakukan left join, tentukan parameter `how`

In [ ]:
df_2015.join(df_dam, on=['province'], how='left').show(40)

Update history

2302.1956
* _Translate dan lengkapi penjelasan_
* _Menambahkan bab transformasi DataFrame_
* _Code cleanup_
